# Trả lời câu hỏi `Dự đoán một ngày nào đó có mưa không tại một tỉnh thành cụ thể?`

## Hiểu dữ liệu
- Thông qua việc tiền xử lý và khám phá dữ liệu, ta biết được rằng khi một ngày nào đó có mưa hay không thì giá trị của Precip > 0, Precipprob = 100%, Precipcover>0, Preciptype sẽ xuất hiện 'rain' hoặc rỗng
- Vì vậy: chúng ta không chọn đặc trưng là những thuộc tính này

## Import

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

## Lấy dữ liệu

In [2]:
# Load dữ liệu
data1 = pd.read_csv('../../../data/clean_data/BenTre.csv')
data2 = pd.read_csv('../../../data/clean_data/DaNang.csv')
data3 = pd.read_csv('../../../data/clean_data/HaNoi.csv')
data4 = pd.read_csv('../../../data/clean_data/HoChiMinh.csv')
data5 = pd.read_csv('../../../data/clean_data/LamDong.csv')
data6 = pd.read_csv('../../../data/clean_data/LangSon.csv')
data7 = pd.read_csv('../../../data/clean_data/NgheAn.csv')
data8 = pd.read_csv('../../../data/clean_data/SonLa.csv')
data = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8 ], ignore_index=True)

# Chuyển cột 'Datetime' thành kiểu datetime nếu chưa
data['Datetime'] = pd.to_datetime(data['Datetime'], errors='coerce')

data.head()

,Address,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Dew,Humidity,Precip,Precipprob,...,Windgust,Windspeed,Winddir,Pressure,Cloudcover,Visibility,Solarradiation,Solarenergy,Uvindex,Moonphase
0,Bến Tre,2019-01-01,1546275600,27.7,24.0,25.8,19.2,66.9,2.7,100.0,...,36.4,18.4,3.2,1012.5,57.3,10.9,81.2,7.0,3.0,0.83
1,Bến Tre,2019-01-02,1546362000,25.0,23.5,24.1,20.8,81.5,26.5,100.0,...,56.9,14.4,3.5,1012.0,66.1,9.0,38.0,3.2,1.0,0.87
2,Bến Tre,2019-01-03,1546448400,28.3,23.7,26.2,21.8,77.1,0.1,100.0,...,62.6,17.8,47.6,1012.0,65.6,9.7,109.7,9.5,5.0,0.90
3,Bến Tre,2019-01-04,1546534800,32.1,25.5,28.3,22.6,72.4,1.2,100.0,...,60.1,30.3,88.8,1012.6,50.4,10.7,196.2,17.0,7.0,0.94
4,Bến Tre,2019-01-05,1546621200,32.6,25.0,28.2,22.0,70.2,0.0,0.0,...,45.4,27.3,97.8,1012.6,36.5,10.9,223.0,19.2,8.0,0.97


## Tạo thêm cột Rain trong Dataframe, nếu ngày đó mưa thì là 1, nếu ngày đó không mưa thì là 0

In [3]:
# Tạo cột mục tiêu nhị phân từ `Precipprob`
data['Rain'] = (data['Precipprob'] > 50).astype(int)
data.head()

,Address,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Dew,Humidity,Precip,Precipprob,...,Windspeed,Winddir,Pressure,Cloudcover,Visibility,Solarradiation,Solarenergy,Uvindex,Moonphase,Rain
0,Bến Tre,2019-01-01,1546275600,27.7,24.0,25.8,19.2,66.9,2.7,100.0,...,18.4,3.2,1012.5,57.3,10.9,81.2,7.0,3.0,0.83,1
1,Bến Tre,2019-01-02,1546362000,25.0,23.5,24.1,20.8,81.5,26.5,100.0,...,14.4,3.5,1012.0,66.1,9.0,38.0,3.2,1.0,0.87,1
2,Bến Tre,2019-01-03,1546448400,28.3,23.7,26.2,21.8,77.1,0.1,100.0,...,17.8,47.6,1012.0,65.6,9.7,109.7,9.5,5.0,0.90,1
3,Bến Tre,2019-01-04,1546534800,32.1,25.5,28.3,22.6,72.4,1.2,100.0,...,30.3,88.8,1012.6,50.4,10.7,196.2,17.0,7.0,0.94,1
4,Bến Tre,2019-01-05,1546621200,32.6,25.0,28.2,22.0,70.2,0.0,0.0,...,27.3,97.8,1012.6,36.5,10.9,223.0,19.2,8.0,0.97,0


## Chia dữ liệu thành 3 tập train, valid, test

In [4]:
# Chia dữ liệu thành tập train (2019-2020), test (2021-2022), valid (>2023)
train_data = data[(data['Datetime'].dt.year >= 2019) & (data['Datetime'].dt.year <= 2020)].copy()
display(train_data)
test_data = data[(data['Datetime'].dt.year >= 2021) & (data['Datetime'].dt.year <= 2022)].copy()
display(test_data)
valid_data = data[(data['Datetime'].dt.year >= 2023)].copy()
display(valid_data)

,Address,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Dew,Humidity,Precip,Precipprob,...,Windspeed,Winddir,Pressure,Cloudcover,Visibility,Solarradiation,Solarenergy,Uvindex,Moonphase,Rain
0,Bến Tre,2019-01-01,1546275600,27.7,24.0,25.8,19.2,66.9,2.7,100.0,...,18.4,3.2,1012.5,57.3,10.9,81.2,7.0,3.0,0.83,1
1,Bến Tre,2019-01-02,1546362000,25.0,23.5,24.1,20.8,81.5,26.5,100.0,...,14.4,3.5,1012.0,66.1,9.0,38.0,3.2,1.0,0.87,1
2,Bến Tre,2019-01-03,1546448400,28.3,23.7,26.2,21.8,77.1,0.1,100.0,...,17.8,47.6,1012.0,65.6,9.7,109.7,9.5,5.0,0.90,1
3,Bến Tre,2019-01-04,1546534800,32.1,25.5,28.3,22.6,72.4,1.2,100.0,...,30.3,88.8,1012.6,50.4,10.7,196.2,17.0,7.0,0.94,1
4,Bến Tre,2019-01-05,1546621200,32.6,25.0,28.2,22.0,70.2,0.0,0.0,...,27.3,97.8,1012.6,36.5,10.9,223.0,19.2,8.0,0.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13515,Sơn La,2020-12-27,1609002000,25.9,9.2,17.8,14.5,82.8,0.0,0.0,...,7.9,107.3,1014.4,38.6,20.0,151.5,13.0,6.0,0.41,0
13516,Sơn La,2020-12-28,1609088400,25.9,14.8,19.3,15.9,82.3,0.1,100.0,...,10.8,8.1,1012.6,53.0,17.5,160.3,14.0,7.0,0.44,1
13517,Sơn La,2020-12-29,1609174800,28.5,13.2,20.1,14.8,74.3,0.0,0.0,...,9.4,67.3,1013.1,15.5,20.0,203.4,17.6,8.0,0.47,0
13518,Sơn La,2020-12-30,1609261200,19.1,11.8,16.1,10.7,71.1,0.0,0.0,...,14.4,126.0,1021.8,63.3,20.0,145.3,12.6,6.0,0.50,0


,Address,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Dew,Humidity,Precip,Precipprob,...,Windspeed,Winddir,Pressure,Cloudcover,Visibility,Solarradiation,Solarenergy,Uvindex,Moonphase,Rain
731,Bến Tre,2021-01-01,1609434000,28.5,22.7,25.1,17.7,64.0,0.0,0.0,...,14.9,17.0,1011.8,70.2,11.1,173.0,14.8,7.0,0.57,0
732,Bến Tre,2021-01-02,1609520400,29.1,21.5,24.9,18.4,67.4,0.0,0.0,...,13.8,0.4,1011.0,72.7,10.9,167.0,14.5,7.0,0.61,0
733,Bến Tre,2021-01-03,1609606800,31.0,22.0,26.2,21.2,74.5,0.2,100.0,...,11.2,347.2,1010.3,55.0,10.8,198.9,17.2,8.0,0.64,1
734,Bến Tre,2021-01-04,1609693200,31.1,23.5,27.1,22.6,77.7,0.6,100.0,...,12.2,50.3,1009.8,57.9,10.4,147.5,12.7,5.0,0.68,1
735,Bến Tre,2021-01-05,1609779600,30.1,23.9,26.7,23.2,81.9,5.0,100.0,...,12.0,42.5,1010.3,78.0,10.3,164.8,14.4,7.0,0.72,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14245,Sơn La,2022-12-27,1672074000,20.6,10.6,15.4,12.6,84.1,0.1,100.0,...,11.5,135.6,1018.2,74.9,20.7,113.6,9.7,5.0,0.13,1
14246,Sơn La,2022-12-28,1672160400,16.9,10.6,14.0,12.9,93.0,11.0,100.0,...,14.4,136.6,1019.6,89.2,18.5,67.5,5.7,4.0,0.16,1
14247,Sơn La,2022-12-29,1672246800,12.6,8.0,10.4,8.5,88.3,2.7,100.0,...,10.8,136.1,1024.1,97.4,17.2,60.0,5.1,3.0,0.20,1
14248,Sơn La,2022-12-30,1672333200,13.5,7.9,10.3,7.5,83.4,0.0,0.0,...,10.8,130.4,1025.2,84.7,23.2,76.7,6.5,4.0,0.25,0


,Address,Datetime,DatetimeEpoch,Tempmax,Tempmin,Temp,Dew,Humidity,Precip,Precipprob,...,Windspeed,Winddir,Pressure,Cloudcover,Visibility,Solarradiation,Solarenergy,Uvindex,Moonphase,Rain
1461,Bến Tre,2023-01-01,1672506000,29.8,22.5,25.6,19.7,70.7,0.1,100.0,...,13.0,7.6,1012.9,51.6,9.9,134.5,11.6,6.0,0.30,1
1462,Bến Tre,2023-01-02,1672592400,30.5,22.5,25.9,21.9,79.5,0.9,100.0,...,8.5,316.2,1013.4,61.4,10.4,174.0,15.2,8.0,0.34,1
1463,Bến Tre,2023-01-03,1672678800,32.0,23.0,27.1,22.3,76.6,4.1,100.0,...,12.7,19.2,1012.5,55.4,10.6,225.8,19.5,8.0,0.37,1
1464,Bến Tre,2023-01-04,1672765200,30.9,23.5,26.8,20.4,68.9,1.8,100.0,...,16.9,20.4,1011.9,54.7,10.6,135.8,11.7,6.0,0.41,1
1465,Bến Tre,2023-01-05,1672851600,27.2,24.0,25.5,20.0,71.9,0.1,100.0,...,20.4,16.4,1011.4,75.8,10.8,23.3,2.1,1.0,0.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14611,Sơn La,2023-12-28,1703696400,24.2,9.5,15.6,10.8,75.9,0.0,0.0,...,7.9,138.3,1020.6,9.6,21.9,202.2,17.5,7.0,0.54,0
14612,Sơn La,2023-12-29,1703782800,24.9,11.1,16.9,12.5,77.6,0.0,0.0,...,8.6,130.1,1019.0,43.4,21.5,173.2,14.8,7.0,0.57,0
14613,Sơn La,2023-12-30,1703869200,25.7,12.7,18.2,13.7,77.4,0.0,0.0,...,8.6,132.9,1017.2,38.8,21.5,169.6,14.5,7.0,0.60,0
14614,Sơn La,2023-12-31,1703955600,26.2,13.4,18.7,14.0,76.6,0.1,100.0,...,10.8,138.8,1016.6,43.4,21.8,164.4,14.2,7.0,0.64,1


## Tính hệ số tương quan để chọn đặc trưng

In [5]:
# Tính hệ số tương quan giữa các cột số và cột mục tiêu 'Rain'
correlation = data.corr()['Rain'].sort_values(ascending=False)
print(correlation)

Precipprob        1.000000
Rain              1.000000
Humidity          0.530599
Precipcover       0.468178
Cloudcover        0.446564
Dew               0.275494
Precip            0.216921
Tempmin           0.140102
Winddir           0.121091
Windgust          0.066089
DatetimeEpoch     0.047554
Temp              0.034923
Moonphase         0.017115
Windspeed         0.000850
Tempmax          -0.050351
Visibility       -0.102465
Pressure         -0.163866
Uvindex          -0.249350
Solarradiation   -0.286382
Solarenergy      -0.286481
Name: Rain, dtype: float64


## Chọn đặc trưng và chuẩn hóa dữ liệu

Dựa vào các hệ số tương quan ở trên, chọn các đặc trưng có hệ số tương quan với giá trị tuyệt đối lớn hơn 0.28:
- Humidity
- Cloudcover
- Solarenergy

Lưu ý: Không chọn các đặc trưng Precipprob, Rain, Precipcover như đã đề cập ở trên.

In [6]:
# Chọn đặc trưng và chuẩn hóa dữ liệu cho tập train, test, valid
numeric_features = [ 'Humidity','Cloudcover', 'Solarenergy']

scaler = StandardScaler()

train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])
display(train_data[numeric_features].head())

valid_data[numeric_features] = scaler.transform(valid_data[numeric_features]) 
display(valid_data[numeric_features].head())

test_data[numeric_features] = scaler.transform(test_data[numeric_features]) 
display(test_data[numeric_features].head())

,Humidity,Cloudcover,Solarenergy
0,-1.356082,-0.306315,-1.529781
1,0.221170,0.084834,-2.136547
2,-0.254166,0.062610,-1.130593
3,-0.761912,-0.613012,0.066972
4,-0.999580,-1.230851,0.418257


,Humidity,Cloudcover,Solarenergy
1461,-0.945565,-0.559674,-0.795275
1462,0.005108,-0.124075,-0.220444
1463,-0.308182,-0.390768,0.466160
1464,-1.140020,-0.421882,-0.779307
1465,-0.815927,0.515988,-2.312190


,Humidity,Cloudcover,Solarenergy
731,-1.669372,0.267075,-0.284314
732,-1.302067,0.378197,-0.332216
733,-0.535047,-0.408548,0.098907
734,-0.189348,-0.279646,-0.619632
735,0.264382,0.613776,-0.348184


In [7]:
# Chia dữ liệu cho từng biến
X_data = data[numeric_features]
y_data = data['Rain']
X_train = train_data[numeric_features]
y_train = train_data['Rain']
X_test = test_data[numeric_features]
y_test = test_data['Rain']
X_valid = valid_data[numeric_features]
y_valid = valid_data['Rain']

## Huấn luyện mô hình bằng tập train và kiểm tra mô hình bằng tập test

In [8]:
# Danh sách các mô hình
models = [LogisticRegression(max_iter=500, C=0.01, penalty='l2'),
          KNeighborsClassifier(n_neighbors=5),
          GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)]

def train(models: list):
    '''
    models:list -> y_pred_lists: list, trained_models: list
    Nhận vào danh sách các models và thực hiện luyện luyện cho từng mô hình, trả về kết quả dự đoán của từng mô hình 
    trong y_pred_lists và danh sách các mô hình đã được huấn luyện trong trained_models
    '''
    y_pred_list = []
    trained_models = []
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_list.append(y_pred)
        trained_models.append(model)
    
    return y_pred_list, trained_models
def report(y_preds: list, trained_models: list):
    '''
    y_preds: list, trained_models: list -> None
    Nhận vào danh sách kết quả dự đoán và mô hình đã được huấn luyện, trả về báo cáo về các độ đo 
    như precision, recall, accuracy
    '''
    for i in range(len(y_preds)):
        print("Classification Report:\n", classification_report(y_test, y_preds[i]))
        accuracy = accuracy_score(y_test, y_preds[i])
        print(f"Tỉ lệ chính xác của mô hình {trained_models[i].__class__.__name__} trong tập test: {accuracy * 100:.2f}%\n")

In [9]:
# Test
y_pred_list, trained_models = train(models)
report(y_pred_list, trained_models)

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.52      0.61      1714
           1       0.82      0.93      0.87      4126

    accuracy                           0.81      5840
   macro avg       0.79      0.72      0.74      5840
weighted avg       0.80      0.81      0.80      5840

Tỉ lệ chính xác của mô hình LogisticRegression trong tập test: 80.86%

Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.55      0.58      1714
           1       0.82      0.86      0.84      4126

    accuracy                           0.77      5840
   macro avg       0.72      0.70      0.71      5840
weighted avg       0.76      0.77      0.76      5840

Tỉ lệ chính xác của mô hình KNeighborsClassifier trong tập test: 76.80%

Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.58      0.62      1714
           1       0.

## Kiểm tra lại lần nữa bằng tập valid

In [10]:
def valid(models: list):
    '''
    models: list -> None
    Nhận vào các mô hình đã huấn luyện và đánh giá kết quả với tập valid dựa trên độ đo accuracy
    '''
    for model in models:
        y_pred_2023 = model.predict(X_valid)
        # Tính tỉ lệ chính xác
        accuracy_2023 = accuracy_score(y_valid, y_pred_2023)
        print(f"Tỉ lệ chính xác của mô hình {model.__class__.__name__} trong tập valid: {accuracy_2023 * 100:.2f}%")
        print()

valid(trained_models)

Tỉ lệ chính xác của mô hình LogisticRegression trong tập valid: 81.59%

Tỉ lệ chính xác của mô hình KNeighborsClassifier trong tập valid: 77.77%

Tỉ lệ chính xác của mô hình GradientBoostingClassifier trong tập valid: 76.98%



## Đánh giá lại bằng cross-validation

In [11]:
def cross_valid(models: list):
    for model in models:
        print(model.__repr__())
        cv_scores = cross_val_score(model, X_data, y_data, cv=5, scoring='accuracy') 
        print("Cross-Validation Scores:", cv_scores) 
        print("Mean Accuracy:", cv_scores.mean())
        print()
cross_valid(models)

LogisticRegression(C=0.01, max_iter=500)
Cross-Validation Scores: [0.79343365 0.75367773 0.85254875 0.79575778 0.79028395]
Mean Accuracy: 0.7971403740230907

KNeighborsClassifier()
Cross-Validation Scores: [0.75820793 0.73725624 0.81628464 0.7865207  0.75401984]
Mean Accuracy: 0.7704578715063745

GradientBoostingClassifier(random_state=42)
Cross-Validation Scores: [0.80369357 0.78515224 0.84570647 0.79952104 0.7817311 ]
Mean Accuracy: 0.8031608830947349



## Cách sử dụng mô hình để trả lời câu hỏi được đặt ra:
Đưa vào các đặc trưng của một ngày nào đó mà bạn muốn dữ đoán, ở đây có 3 mô hình, nếu 2 trong 3 mô hình dự đoán mưa thì sẽ là mưa, không mưa thì sẽ không mưa

In [12]:
# Dữ liệu đầu vào ví dụ
input_data = [[82.9, 48.5, 15.3]]
feature_names = ['Humidity','Cloudcover', 'Solarenergy']
df = pd.DataFrame(input_data, columns=feature_names)
# Dự đoán 
for model in trained_models:
    y = model.predict(df)
    if y[0] == 1:
        is_rain = 'mưa'
    else:
        is_rain = 'không mưa'
    print(f"Mô hình {model.__class__.__name__} dự đoán {is_rain}.\n")

Mô hình LogisticRegression dự đoán mưa.

Mô hình KNeighborsClassifier dự đoán mưa.

Mô hình GradientBoostingClassifier dự đoán không mưa.



## Kết luận:
- Kết quả dự đoán của 3 mô hình LogisticRegression, KNeighborsClassifier, GradientBoostingClassifier có kết quả tương đối bằng nhau, đạt độ chính xác khoảng 79%.
- Tính thực dụng của mô hình không cao, để sử dụng mô hình để dự đoán một ngày nào đó có mưa hay không thì cần phải lấy được thông số về các đặc trưng như 'Humidity','Cloudcover', 'Solarenergy', các đặc trưng này cần có công cụ đo đặc biệt.
- Việc muốn ứng dụng mô hình cho việc sử dụng cá nhân thì mỗi cá nhân có thể lấy các đặc trưng này thông qua API mà nhóm đã sử dụng để lấy dữ liệu, tuy nhiên API này cũng cung cấp dự đoán về một ngày nào đó có mưa hay không, nên mô hình này chỉ nên sử dụng khi bạn không tin tưởng việc dự đoán của API và muốn tự tạo mô hình để dự đoán một ngày nào đó có mưa tại Việt Nam không